## The Movies Database

Given is the diagram of movies database consisting of 3 tables - Movies, Halls and Tickets

* Movie: This table has 4 columns - Movie_id (unique identifier for each Movie specific to language), Movie_name (Name of the movie), Language (Language of the Movie), Rating (Average rating given by viewers)
* Hall: This table has 3 columns - Hall_id (unique identifer for each Movie Hall), Hall_name (Name of the hall), Seating_capacity (maximum ticketed seats available in the hall)
* Ticket: This table has 3 columns - Movie_id (unique identifier for each Movie specific to language), Hall_id (unique identifer for each Movie Hall), Tickets_sold (number of tickets sold for the given Movie at the given Hall)

<img src="../../../images/movies_db.PNG" style="width: 65vw;"> <br>

<b>Tasks:</b>
1. Create an empty database named 'moviesdb' using sqlite terminal.
2. Create empty tables with connected relationships among the three tables as shown in above diagram. Using foreign key constraints and enforce referential integrity.
3. Extract the data from the three tables, links of which are provided in the code block. Load them into a dataframe and then into the table.
4. Write a query to extract data of which movie ran at over 80% of seating capacity and at which hall this was achieved. The query should extract all combinations of movie and hall with over 80% capacity.

In [1]:
movies_data_link = "../../../data/movie.csv"
halls_data_link = "../../../data/hall.csv"
tickets_data_link = "../../../data/ticket.csv"

In [2]:
!pip install tabulate

In [3]:
import sqlite3
import csv
import pandas as pd
from tabulate import tabulate

# Connecting to the database
vincon = sqlite3.connect('moviesdb.db')
vincon.execute("PRAGMA foreign_keys = ON")
vincur = vincon.cursor()


In [5]:
# creating movies table
try:
    vincur.execute("""CREATE TABLE movies(
    movie_id    INTEGER PRIMARY KEY, 
    movie_name  TEXT,
    language   TEXT,
    rating     INTEGER
    );""")
except Exception as e:
    print('Error:',e)


In [6]:
# creating halls table
try:
    vincur.execute("""CREATE TABLE halls(
    hall_id    INTEGER PRIMARY KEY, 
    hall_name   TEXT,
    seating_capacity     INTEGER
    );""")
except Exception as e:
    print('Error:',e)


In [7]:
# creating tickets table
try :
    vincur.execute('''CREATE TABLE tickets(
    movie_id     INTEGER ,
    hall_id      INTEGER ,
    tickets_sold  INTEGER    NON NULL,
    FOREIGN KEY(movie_id) REFERENCES movie(movie_id),
    FOREIGN KEY(hall_id) REFERENCES hall(hall_id)
    );''')
except Exception as e :
    print('Exception occurred :',e)

In [12]:
# Reading data into tables
moviesdf = pd.read_csv(movies_data_link)
hallsdf = pd.read_csv(halls_data_link)
ticketsdf = pd.read_csv(tickets_data_link)

moviesdf.insert(loc=0, column='Movie_Id', value=[24, 56, 22, 63, 43, 33, 11])

try:
    moviesdf.to_sql(name='movies',con=vincon,if_exists='replace',index=False)
    hallsdf.to_sql(name='halls',con=vincon,if_exists='replace',index=False)
    ticketsdf.to_sql(name='tickets',con=vincon,if_exists='replace',index=False)
except Exception as e :
    print(e)


In [13]:
vincur.execute("""
SELECT * FROM movies
""")
vin_movie = vincur.fetchall()
headers_movie = ['Movie_Id', 'Movie_name', 'Language', 'Rating']
print(tabulate(vin_movie, headers_movie))

  Movie_Id  Movie_name          Language      Rating
----------  ------------------  ----------  --------
        24  Lakewalker          Mandarin         6.4
        56  Ramen loving Ronin  Japanese         8.1
        22  Senjiruven          Tamil            7.3
        63  The Unimaginable    English          7.9
        43  La Belle            French           6.8
        33  Kobali              Hindi            6.9
        11  Kobali              Telugu           6.9


In [14]:
vincur.execute("""
SELECT * FROM halls
""")
vin_hall = vincur.fetchall()
headers_hall = ['Hall_Id', 'Hall_name', 'Seating_capacity']
print(tabulate(vin_hall, headers_hall))

  Hall_Id  Hall_name                   Seating_capacity
---------  ------------------------  ------------------
        1  Princessville Cinema                     150
        2  Wang's Town Cinema                       150
        3  Ardour Movie Hall                        150
        4  VMC Hollowville                          150
        5  VMC Frocksburry                          150
        6  VMC Browns                               150
        7  VMC Quasiland                            150
        8  VMC Goodborough                          150
        9  Showtime Browns                          120
       10  Showtime Nathan Square                   120
       11  Showtime Rivermoore                      120
       12  Showtime Shwimm's Market                 120
       13  Cangshin Boxes                            90
       14  Wurchester Cinema                        150
       15  Quagmire Movie Hall                      150


In [15]:
vincur.execute("""
SELECT * FROM tickets
""")
vin_ticket = vincur.fetchall()
headers_ticket = ['Movie_Id', 'Hall_Id', 'Tickets_sold']
print(tabulate(vin_ticket, headers_ticket))

  Movie_Id    Hall_Id    Tickets_sold
----------  ---------  --------------
        63         14             100
        56          2             106
        24         13              34
        63          1             136
        11         15             114
        11         11              47
        22          6              73
        24         15              25
        33         15              89
        22         14              87
        11         10              86
        11          1             121
        11          8             104
        24          9              14
        56          5             109
        33         13              76
        33          6              29
        43         11              88
        22         10              82
        33          2              58
        56         15             101
        56         12              27
        33         12             116
        63          4              94
        63  